In [1]:
# Hey , this would be the final script that extracts all the data I need for all the matches
# for all the seasons for the italian championship.
# Further improuvements of this project would be to iterate over all the championships
# fly down for now
import numpy as np
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import time
import random

In [2]:
# after doing some magic it extract some beautiful and clean json
def clean_json_data(json_string_with_data):
    data = []
    for message in json_string_with_data:
        ind_start = message.index("('")+2
        ind_end = message.index("')")
        json_data = message[ind_start:ind_end]
        json_data = json_data.encode('utf8').decode('unicode_escape')
        data.append(json.loads(json_data))
    return data

In [3]:
# Useful constants
base_url_series = 'https://understat.com/league/'
base_url_matches = 'https://understat.com/match/'
series = ['Serie_A']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019']
shotsDF = pd.DataFrame()
roastersDF = pd.DataFrame()
n_iterations = len(series)*len(seasons)*380
counter = 0

In [4]:
# let's do the trick
# The first iteration is for all the serie in series, and the second is 
# for all the season in seasons, ask for the list of matches
# then, for each match in matches, ask for the data related to that match
for serie in series:
    for season in seasons:
        url = base_url_series+serie+'/'+season+'/'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "lxml")
        # extract fom this url the list of matches for the selected season for the selected serie
        # simply copy paste from the previous example and filter all the bullshit
        scripts = soup.find_all('script')
        json_string_with_data = []
        # don't need all the data collecetd in the previous example
        for el in scripts:
            if 'datesData' in el.text:
                json_string_with_data.append(el.text.strip())
        # clean the data and use those
        data = clean_json_data(json_string_with_data)
        columns = list(data[0][0].keys())
        values = []
        for i, item in enumerate(data[0]):
            values.append(list(data[0][i].values()))
        # for this loop create a partial data frame
        partial_df = pd.DataFrame(values, columns=columns)
        partial_df = partial_df[partial_df.isResult == True]
        matches = partial_df['id']
        # now we do have a eautiful formatted list of all the matches occurred in this iteration
        # let's iterate over each match and ask for the data of that match
        for match in matches:
            url = base_url_matches+match+'/'
            res = requests.get(url)
            soup = BeautifulSoup(res.content, "lxml")
            # let's copy paste from the other tutorial in which from a selected match we extract 
            # all the events that have been occurred
            scripts = soup.find_all('script')
            json_string_with_data = []
            for el in scripts:
                if 'shotsData' in el.text:
                    json_string_with_data.append(el.text.strip())
                if 'rostersData' in el.text:
                    json_string_with_data.append(el.text.strip())
            # clean the data
            data = clean_json_data(json_string_with_data)
            values = []
            for key in data[1]['h'].keys():
                columns = list(data[1]['h'][key].keys())
                values.append(list(data[1]['h'][key].values()))
            for key in data[1]['a'].keys():
                values.append(list(data[1]['a'][key].values()))
            # create a partial dataframe for the shotsData
            # TODO: want to add tre columns to each data frame regarding the 
            # match id, season year and season name/code
            shotsData = pd.DataFrame(values, columns=columns)
            shotsData.insert(0, 'match_id', match)
            shotsData.insert(1, 'season_year', season)
            shotsData.insert(2, 'championship_name', serie)
            shotsDF = shotsDF.append(shotsData, ignore_index=True)
            # extract also the data for the penalities
            values = []
            columns = list(data[0]['h'][0].keys())
            for key in data[0].keys():
                for i, event in enumerate(data[0][key]):
                    values.append(list(data[0][key][i].values()))
            # create a partial dataframe for the roastersData
            roastersData = pd.DataFrame(values, columns=columns)
            roastersData.insert(1, 'championship_name', serie)
            roastersDF = roastersDF.append(roastersData, ignore_index=True)
            counter+=1
            print('iteration ', counter, 'of ',n_iterations, ' match', match)
            time.sleep(1)
print(shotsDF.head())
print(roastersDF.head())
# save this data frames in a csv file
shotsDF.to_csv('shots_df.csv')
roastersDF.to_csv('roasters_df.csv')
            

iteration  1 of  2280  match 5149
iteration  2 of  2280  match 5150
iteration  3 of  2280  match 5151
iteration  4 of  2280  match 5152
iteration  5 of  2280  match 5153
iteration  6 of  2280  match 5154
iteration  7 of  2280  match 5155
iteration  8 of  2280  match 5156
iteration  9 of  2280  match 5157
iteration  10 of  2280  match 5158
iteration  11 of  2280  match 5109
iteration  12 of  2280  match 5110
iteration  13 of  2280  match 5111
iteration  14 of  2280  match 5112
iteration  15 of  2280  match 5113
iteration  16 of  2280  match 5114
iteration  17 of  2280  match 5115
iteration  18 of  2280  match 5116
iteration  19 of  2280  match 5117
iteration  20 of  2280  match 5118
iteration  21 of  2280  match 5119
iteration  22 of  2280  match 5120
iteration  23 of  2280  match 5121
iteration  24 of  2280  match 5122
iteration  25 of  2280  match 5123
iteration  26 of  2280  match 5124
iteration  27 of  2280  match 5125
iteration  28 of  2280  match 5126
iteration  29 of  2280  match

iteration  232 of  2280  match 4948
iteration  233 of  2280  match 4949
iteration  234 of  2280  match 4950
iteration  235 of  2280  match 4951
iteration  236 of  2280  match 4952
iteration  237 of  2280  match 4953
iteration  238 of  2280  match 4954
iteration  239 of  2280  match 4955
iteration  240 of  2280  match 4956
iteration  241 of  2280  match 4881
iteration  242 of  2280  match 4882
iteration  243 of  2280  match 4883
iteration  244 of  2280  match 4884
iteration  245 of  2280  match 4885
iteration  246 of  2280  match 4886
iteration  247 of  2280  match 4887
iteration  248 of  2280  match 4888
iteration  249 of  2280  match 4889
iteration  250 of  2280  match 4890
iteration  251 of  2280  match 4891
iteration  252 of  2280  match 4892
iteration  253 of  2280  match 4893
iteration  254 of  2280  match 4894
iteration  255 of  2280  match 4895
iteration  256 of  2280  match 4896
iteration  257 of  2280  match 4897
iteration  258 of  2280  match 4898
iteration  259 of  2280  mat

iteration  462 of  2280  match 632
iteration  463 of  2280  match 633
iteration  464 of  2280  match 634
iteration  465 of  2280  match 635
iteration  466 of  2280  match 636
iteration  467 of  2280  match 637
iteration  468 of  2280  match 638
iteration  469 of  2280  match 639
iteration  470 of  2280  match 640
iteration  471 of  2280  match 641
iteration  472 of  2280  match 642
iteration  473 of  2280  match 643
iteration  474 of  2280  match 644
iteration  475 of  2280  match 645
iteration  476 of  2280  match 646
iteration  477 of  2280  match 647
iteration  478 of  2280  match 648
iteration  479 of  2280  match 649
iteration  480 of  2280  match 650
iteration  481 of  2280  match 651
iteration  482 of  2280  match 652
iteration  483 of  2280  match 653
iteration  484 of  2280  match 654
iteration  485 of  2280  match 655
iteration  486 of  2280  match 656
iteration  487 of  2280  match 657
iteration  488 of  2280  match 658
iteration  489 of  2280  match 659
iteration  490 of  2

iteration  697 of  2280  match 867
iteration  698 of  2280  match 868
iteration  699 of  2280  match 869
iteration  700 of  2280  match 870
iteration  701 of  2280  match 871
iteration  702 of  2280  match 872
iteration  703 of  2280  match 873
iteration  704 of  2280  match 874
iteration  705 of  2280  match 875
iteration  706 of  2280  match 876
iteration  707 of  2280  match 877
iteration  708 of  2280  match 878
iteration  709 of  2280  match 879
iteration  710 of  2280  match 880
iteration  711 of  2280  match 881
iteration  712 of  2280  match 882
iteration  713 of  2280  match 883
iteration  714 of  2280  match 884
iteration  715 of  2280  match 885
iteration  716 of  2280  match 886
iteration  717 of  2280  match 887
iteration  718 of  2280  match 888
iteration  719 of  2280  match 889
iteration  720 of  2280  match 890
iteration  721 of  2280  match 891
iteration  722 of  2280  match 892
iteration  723 of  2280  match 893
iteration  724 of  2280  match 894
iteration  725 of  2

iteration  929 of  2280  match 3095
iteration  930 of  2280  match 3096
iteration  931 of  2280  match 3097
iteration  932 of  2280  match 3098
iteration  933 of  2280  match 3100
iteration  934 of  2280  match 3101
iteration  935 of  2280  match 3102
iteration  936 of  2280  match 3099
iteration  937 of  2280  match 3103
iteration  938 of  2280  match 3104
iteration  939 of  2280  match 3520
iteration  940 of  2280  match 3521
iteration  941 of  2280  match 3522
iteration  942 of  2280  match 3524
iteration  943 of  2280  match 3525
iteration  944 of  2280  match 3526
iteration  945 of  2280  match 3527
iteration  946 of  2280  match 3528
iteration  947 of  2280  match 3529
iteration  948 of  2280  match 3530
iteration  949 of  2280  match 3531
iteration  950 of  2280  match 3532
iteration  951 of  2280  match 3533
iteration  952 of  2280  match 3534
iteration  953 of  2280  match 3535
iteration  954 of  2280  match 3536
iteration  955 of  2280  match 3537
iteration  956 of  2280  mat

iteration  1153 of  2280  match 7510
iteration  1154 of  2280  match 7511
iteration  1155 of  2280  match 7512
iteration  1156 of  2280  match 7513
iteration  1157 of  2280  match 7514
iteration  1158 of  2280  match 7516
iteration  1159 of  2280  match 7517
iteration  1160 of  2280  match 7518
iteration  1161 of  2280  match 7525
iteration  1162 of  2280  match 7524
iteration  1163 of  2280  match 7519
iteration  1164 of  2280  match 7522
iteration  1165 of  2280  match 7523
iteration  1166 of  2280  match 7528
iteration  1167 of  2280  match 7526
iteration  1168 of  2280  match 7520
iteration  1169 of  2280  match 7521
iteration  1170 of  2280  match 7530
iteration  1171 of  2280  match 7531
iteration  1172 of  2280  match 7535
iteration  1173 of  2280  match 7536
iteration  1174 of  2280  match 7533
iteration  1175 of  2280  match 7534
iteration  1176 of  2280  match 7537
iteration  1177 of  2280  match 7538
iteration  1178 of  2280  match 7529
iteration  1179 of  2280  match 7532
i

iteration  1375 of  2280  match 7732
iteration  1376 of  2280  match 7731
iteration  1377 of  2280  match 7733
iteration  1378 of  2280  match 7734
iteration  1379 of  2280  match 7738
iteration  1380 of  2280  match 7730
iteration  1381 of  2280  match 7747
iteration  1382 of  2280  match 7739
iteration  1383 of  2280  match 7745
iteration  1384 of  2280  match 7746
iteration  1385 of  2280  match 7740
iteration  1386 of  2280  match 7741
iteration  1387 of  2280  match 7742
iteration  1388 of  2280  match 7743
iteration  1389 of  2280  match 7748
iteration  1390 of  2280  match 7744
iteration  1391 of  2280  match 7755
iteration  1392 of  2280  match 7757
iteration  1393 of  2280  match 7751
iteration  1394 of  2280  match 7749
iteration  1395 of  2280  match 7750
iteration  1396 of  2280  match 7754
iteration  1397 of  2280  match 7758
iteration  1398 of  2280  match 7753
iteration  1399 of  2280  match 7752
iteration  1400 of  2280  match 7759
iteration  1401 of  2280  match 7765
i

iteration  1597 of  2280  match 9652
iteration  1598 of  2280  match 9653
iteration  1599 of  2280  match 9654
iteration  1600 of  2280  match 9663
iteration  1601 of  2280  match 9661
iteration  1602 of  2280  match 9665
iteration  1603 of  2280  match 9660
iteration  1604 of  2280  match 9657
iteration  1605 of  2280  match 9658
iteration  1606 of  2280  match 9662
iteration  1607 of  2280  match 9659
iteration  1608 of  2280  match 9666
iteration  1609 of  2280  match 9664
iteration  1610 of  2280  match 9667
iteration  1611 of  2280  match 9669
iteration  1612 of  2280  match 9675
iteration  1613 of  2280  match 9673
iteration  1614 of  2280  match 9668
iteration  1615 of  2280  match 9670
iteration  1616 of  2280  match 9674
iteration  1617 of  2280  match 9672
iteration  1618 of  2280  match 9676
iteration  1619 of  2280  match 9671
iteration  1620 of  2280  match 9581
iteration  1621 of  2280  match 9683
iteration  1622 of  2280  match 9680
iteration  1623 of  2280  match 9679
i

iteration  1819 of  2280  match 9867
iteration  1820 of  2280  match 9883
iteration  1821 of  2280  match 9886
iteration  1822 of  2280  match 9884
iteration  1823 of  2280  match 9879
iteration  1824 of  2280  match 9878
iteration  1825 of  2280  match 9885
iteration  1826 of  2280  match 9880
iteration  1827 of  2280  match 9881
iteration  1828 of  2280  match 9882
iteration  1829 of  2280  match 9877
iteration  1830 of  2280  match 9895
iteration  1831 of  2280  match 9892
iteration  1832 of  2280  match 9891
iteration  1833 of  2280  match 9896
iteration  1834 of  2280  match 9889
iteration  1835 of  2280  match 9893
iteration  1836 of  2280  match 9894
iteration  1837 of  2280  match 9888
iteration  1838 of  2280  match 9890
iteration  1839 of  2280  match 9887
iteration  1840 of  2280  match 9821
iteration  1841 of  2280  match 9904
iteration  1842 of  2280  match 9897
iteration  1843 of  2280  match 9898
iteration  1844 of  2280  match 9899
iteration  1845 of  2280  match 9900
i

iteration  2037 of  2280  match 13227
iteration  2038 of  2280  match 13223
iteration  2039 of  2280  match 13220
iteration  2040 of  2280  match 13231
iteration  2041 of  2280  match 13229
iteration  2042 of  2280  match 13238
iteration  2043 of  2280  match 13232
iteration  2044 of  2280  match 13233
iteration  2045 of  2280  match 13235
iteration  2046 of  2280  match 13236
iteration  2047 of  2280  match 13237
iteration  2048 of  2280  match 13234
iteration  2049 of  2280  match 13230
iteration  2050 of  2280  match 13240
iteration  2051 of  2280  match 13247
iteration  2052 of  2280  match 13243
iteration  2053 of  2280  match 13244
iteration  2054 of  2280  match 13239
iteration  2055 of  2280  match 13245
iteration  2056 of  2280  match 13246
iteration  2057 of  2280  match 13248
iteration  2058 of  2280  match 13242
iteration  2059 of  2280  match 13241
iteration  2060 of  2280  match 13255
iteration  2061 of  2280  match 13151
iteration  2062 of  2280  match 13250
iteration  2